# Vertex Matching Engine GRPC: MatchRequest (by ID and query vector) and BatchMatchRequest

For PSA and PSC methods of Matching Engine (following code does not support public method)

## Setup

Pip install required packages

In [ ]:
! pip install google-cloud-aiplatform grpcio grpcio-tools

Create directory for required packages and proto specification

In [2]:
! mkdir third_party

In [1]:
%cd third_party

/home/jupyter/third_party


Clone repo with google API dependencies

In [5]:
! git clone https://github.com/googleapis/googleapis.git

Cloning into 'googleapis'...
remote: Enumerating objects: 210572, done.
remote: Counting objects: 100% (11364/11364), done.
remote: Compressing objects: 100% (580/580), done.
remote: Total 210572 (delta 10847), reused 11002 (delta 10773), pack-reused 199208
Receiving objects: 100% (210572/210572), 187.03 MiB | 41.89 MiB/s, done.
Resolving deltas: 100% (179875/179875), done.


Wget the matching engine proto file

In [8]:
! wget https://raw.githubusercontent.com/googleapis/python-aiplatform/main/google/cloud/aiplatform/matching_engine/_protos/match_service.proto

--2023-07-31 22:32:26--  https://raw.githubusercontent.com/googleapis/python-aiplatform/main/google/cloud/aiplatform/matching_engine/_protos/match_service.proto
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7203 (7.0K) [text/plain]
Saving to: ‘match_service.proto’

match_service.proto 100%[===================>]   7.03K  --.-KB/s    in 0.001s  

2023-07-31 22:32:26 (10.8 MB/s) - ‘match_service.proto’ saved [7203/7203]



See proto file for request parameters

In [11]:
! cat match_service.proto

syntax = "proto3";

package google.cloud.aiplatform.container.v1;

import "google/rpc/status.proto";

// MatchService is a Google managed service for efficient vector similarity
// search at scale.
service MatchService {
  // Returns the nearest neighbors for the query. If it is a sharded
  // deployment, calls the other shards and aggregates the responses.
  rpc Match(MatchRequest) returns (MatchResponse) {}

  // Returns the nearest neighbors for batch queries. If it is a sharded
  // deployment, calls the other shards and aggregates the responses.
  rpc BatchMatch(BatchMatchRequest) returns (BatchMatchResponse) {}

  // Looks up the embeddings.
  rpc BatchGetEmbeddings(BatchGetEmbeddingsRequest)
      returns (BatchGetEmbeddingsResponse) {}
}

// Parameters for a match query.
message MatchRequest {
  // The ID of the DeploydIndex that will serve the request.
  // This MatchRequest is sent to a specific IndexEndpoint of the Control API,
  // as per the IndexEndpoint.network. That Ind

Compile the proto file

In [2]:
! python -m grpc_tools.protoc -I=. --proto_path=./googleapis --python_out=. --grpc_python_out=. match_service.proto

/opt/conda/envs/vme/lib/python3.9/site-packages/grpc_tools/protoc.py:21: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


## Required Input

In [ ]:
# Set IP address for your matching engine index deployment (if using PSA, you can find the IP in the console UI under Vertex AI Matching Engine)
ENDPT_ID=""
# Set deployed index ID (can also be found in the console UI)
DEPLOYED_INDEX_ID=""
# Embedding ID for ANN Match Request
EMBED_ID=""

In [3]:
# # Set IP address for your matching engine index deployment (if using PSA, you can find the IP in the console UI under Vertex AI Matching Engine)
# ENDPT_ID="10.128.15.225"
# # Set deployed index ID (can also be found in the console UI)
# DEPLOYED_INDEX_ID="langchain_vector_store_20230622195444"
# # List of your embedding IDs
# EMBED_IDS=! gsutil ls gs://axel-argolis-usc1-bucket/documents | grep -o -P "(?<=gs:\/\/axel-argolis-usc1-bucket\/documents\/).*"
# EMBED_ID=EMBED_IDS[0]

## MatchRequest with Embedding ID and Returned Embedding

In [5]:
import grpc
import match_service_pb2
import match_service_pb2_grpc

# Set up channel and stub
channel = grpc.insecure_channel("{}:10000".format(ENDPT_ID))
stub = match_service_pb2_grpc.MatchServiceStub(channel)

request = match_service_pb2.MatchRequest(embedding_enabled=True,
                                         num_neighbors=2,
                                         deployed_index_id = DEPLOYED_INDEX_ID,
                                         embedding_id = EMBED_ID)

response = stub.Match(request)
response

neighbor {
  id: "000f1413-52cb-4028-828d-f9a30181cc01"
  distance: 0.999993622303009
}
neighbor {
  id: "4c7cb61e-f191-4c85-9d19-6e3c54f6720d"
  distance: 0.89164894819259644
}
embeddings {
  id: "000f1413-52cb-4028-828d-f9a30181cc01"
  float_val: -0.0285835359
  float_val: -0.0128061399
  float_val: -0.00205812324
  float_val: 0.0312303733
  float_val: 0.00970278122
  float_val: -0.0462790765
  float_val: 0.0104515404
  float_val: 0.00385632878
  float_val: -0.0335025117
  float_val: 0.0156927984
  float_val: 0.0468297601
  float_val: -0.0265430268
  float_val: 0.0431479104
  float_val: 0.014064719
  float_val: -0.0109642055
  float_val: 0.0148787433
  float_val: -0.0460174941
  float_val: -0.037452016
  float_val: 0.00404532626
  float_val: -0.00946229789
  float_val: -0.0453124
  float_val: -0.0322842635
  float_val: 0.00282728462
  float_val: 0.00456633791
  float_val: 0.0482513718
  float_val: -0.0283671375
  float_val: 0.0277476273
  float_val: 0.0222511347
  float_val: -0.00526

## MatchRequest with Query Vector and Returned Embedding

Example query from previous response (feel free to replace with your own query vector)

In [7]:
QUERY = list(response.embeddings[0].float_val)

len(QUERY)

768

In [8]:
import grpc
import match_service_pb2
import match_service_pb2_grpc

# Set up channel and stub
channel = grpc.insecure_channel("{}:10000".format(ENDPT_ID))
stub = match_service_pb2_grpc.MatchServiceStub(channel)

request = match_service_pb2.MatchRequest(embedding_enabled=True,
                                         num_neighbors=2,
                                         deployed_index_id = DEPLOYED_INDEX_ID)
for val in QUERY:
    request.float_val.append(val)

response = stub.Match(request)
response

neighbor {
  id: "000f1413-52cb-4028-828d-f9a30181cc01"
  distance: 0.999993622303009
}
neighbor {
  id: "4c7cb61e-f191-4c85-9d19-6e3c54f6720d"
  distance: 0.89164894819259644
}
embeddings {
  id: "4c7cb61e-f191-4c85-9d19-6e3c54f6720d"
  float_val: -0.0251306966
  float_val: -0.00918449927
  float_val: 0.00869452208
  float_val: 0.030915672
  float_val: 0.0178096313
  float_val: -0.0520065576
  float_val: 0.0205601119
  float_val: 0.016895324
  float_val: -0.0320881419
  float_val: -6.37630801e-05
  float_val: 0.0387363285
  float_val: -0.0234905835
  float_val: 0.0147497654
  float_val: 0.00701288879
  float_val: -0.0177649632
  float_val: -0.00803213753
  float_val: -0.0337616503
  float_val: -0.0371527448
  float_val: 0.00124097837
  float_val: 0.00676558726
  float_val: -0.0448020622
  float_val: -0.0523265339
  float_val: 0.0200031381
  float_val: 0.00376741518
  float_val: 0.0254381616
  float_val: -0.0637677237
  float_val: 0.0328518376
  float_val: 0.0272876285
  float_val: -0.

## BatchMatchRequest with Query Vectors

Using vectors from previous response as query vectors (feel free to change to query vectors that you'd like to test)

In [9]:
QUERY_LIST=[list(response.embeddings[0].float_val), list(response.embeddings[1].float_val)]
# FILTER=

In [10]:
from google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint import MatchNeighbor, Namespace

# Set up channel and stub
channel = grpc.insecure_channel("{}:10000".format(ENDPT_ID))
stub = match_service_pb2_grpc.MatchServiceStub(channel)

# Create the batch match request
batch_request = match_service_pb2.BatchMatchRequest()
batch_request_for_index = (
    match_service_pb2.BatchMatchRequest.BatchMatchRequestPerIndex()
)
batch_request_for_index.deployed_index_id = DEPLOYED_INDEX_ID
b_requests = []
for query in QUERY_LIST:
    request = match_service_pb2.MatchRequest(
        num_neighbors=2,
        deployed_index_id=DEPLOYED_INDEX_ID,
        float_val=query,
        embedding_enabled=False
    )
    # TO-DO: Add filter to example
    # for namespace in FILTER:
    #     restrict = match_service_pb2.Namespace()
    #     restrict.name = namespace.name
    #     restrict.allow_tokens.extend(namespace.allow_tokens)
    #     restrict.deny_tokens.extend(namespace.deny_tokens)
    #     request.restricts.append(restrict)
    b_requests.append(request)

batch_request_for_index.requests.extend(b_requests)
batch_request.requests.append(batch_request_for_index)

# Perform the request
response = stub.BatchMatch(batch_request)

# Wrap the results in MatchNeighbor objects and return
neighbors = [
                [
                    MatchNeighbor(id=neighbor.id, distance=neighbor.distance)
                    for neighbor in embedding_neighbors.neighbor
                ]
                for embedding_neighbors in response.responses[0].responses
            ]
neighbors

[[MatchNeighbor(id='4c7cb61e-f191-4c85-9d19-6e3c54f6720d', distance=0.9999935626983643),
  MatchNeighbor(id='a7adaf30-946f-4a0a-89a7-89680a71e911', distance=0.9381897449493408)],
 [MatchNeighbor(id='000f1413-52cb-4028-828d-f9a30181cc01', distance=0.999993622303009),
  MatchNeighbor(id='4c7cb61e-f191-4c85-9d19-6e3c54f6720d', distance=0.8916489481925964)]]